# Cluster-based Input Weight Initialization for Echo State Networks

This notebook aims to be the supplemental material for the corresponding journal article.

We aim to pre-train the input weight matrix of ESNs using the K-Means algorithm since passing features to the non-linear reservoir of ESNs is closely related to compute the dot product between two vectors.

We use various datasets from https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

In [1]:
import os, sys
cwd = os.getcwd()
module_path = os.path.dirname(cwd)  # target working directory

sys.path = [item for item in sys.path if item != module_path]  # remove module_path from sys.path
sys.path.append(module_path)  # add module_path to sys.path

import time
import glob
import os
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, ParameterGrid, cross_val_score
from sklearn.utils import shuffle
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
from sklearn.cluster import MiniBatchKMeans
from joblib import dump, load
from pyrcn.echo_state_network import SeqToLabelESNClassifier, SeqToSeqESNClassifier
from pyrcn.base import PredefinedWeightsInputToNode, NodeToNode
from pyrcn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error
from pyrcn.model_selection import SequentialSearchCV
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
%matplotlib inline
#Options
plt.rc('image', cmap='RdBu')
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=8)

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker

In [2]:
train = True

## Arabian spoken digit dataset 

(https://archive.ics.uci.edu/ml/datasets/Spoken+Arabic+Digit)

- Contains 6600 training utterances 
- Contains 2200 test utterances

Dataset from 8800(10 digits x 10 repetitions x 88 speakers) time series of 13 Frequency Cepstral
Coefficients (MFCCs) had taken from 44 males and 44 females Arabic native speakers
between the ages 18 and 40 to represent ten spoken Arabic digit.

Each line on the data base represents 13 MFCCs coefficients in the increasing order separated by
spaces. This corresponds to one analysis frame. The 13 Mel Frequency Cepstral Coefficients
(MFCCs) are computed with the following
conditions;
Sampling rate: 11025 Hz, 16 bits
Window applied: hamming
Filter pre-emphasized: $1-0.97^{-1}$

In [ ]:
arab = np.load(r"E:\multivariate_time_series_dataset\numpy\ARAB.npz")
X_train = np.empty(shape=(6600, ), dtype=object)
y_train = np.empty(shape=(6600, ), dtype=object)
X_test = np.empty(shape=(2200, ), dtype=object)
y_test = np.empty(shape=(2200, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(arab['X'], arab['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(arab['Xte'], arab['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(arab['X'], arab['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(arab['Xte'], arab['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)
try:
    sequential_search = load("../sequential_search_arab.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_arab.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_arab_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_arab_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Australian Sign Language signs (High Quality) Data Set 

(https://archive.ics.uci.edu/ml/datasets/Australian+Sign+Language+signs+(High+Quality))

- Contains 1140 training utterances 
- Contains 1425 test utterances

Data was captured using a setup that consisted of:

- Two Fifth Dimension Technologies (5DT) gloves, one right and one left
- Two Ascension Flock-of-Birds magnetic position trackers, one attached to each hand
- A four-port serial card to cope with four data sources
- A PC (128MB RAM, Intel Pentium II 266MHz) was used

In terms of the quality of the data, the Flock system was far superior to the Nintendo system also available from the same donor. Firstly, this was a two-hand system. Secondly, each position tracker provided 6 degrees of freedom - i.e. roll, pitch and yaw as well as x, y and z. The gloves also provided a full five fingers of data. But the big improvements were in resolution - both accuracy and temporal. Position and orientation were defined to 14-bit accuracy, giving position information with a typical positional error less than one centimetre and angle error less than one half of a degree. Finger bend was measured with 8 bits per finger, of which probably 6 bits were usable once the glove was calibrated. The refresh rate of the complete system was close to 100 frames per second; and all signals had significantly less noise than the Nintendo data.

Samples from a single signer (a native Auslan signer) were collected over a period of nine weeks. In total, 27 samples per sign, and a total of 2565 signs were collected. The average length of each sign was approximately 57 frames.

The data was collected from a volunteer native Auslan signer 

In [3]:
aus = np.load(r"E:\multivariate_time_series_dataset\numpy\AUS.npz")
X_train = np.empty(shape=(1140, ), dtype=object)
y_train = np.empty(shape=(1140, ), dtype=object)
X_test = np.empty(shape=(1425, ), dtype=object)
y_test = np.empty(shape=(1425, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(aus['X'], aus['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(aus['Xte'], aus['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(aus['X'], aus['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(aus['Xte'], aus['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_aus.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_aus.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [6]:
kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 1600 - 400), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 1600,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_aus_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_aus_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 242.435190
Init 2/200 with method: k-means++


C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)
C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


Inertia for init 2/200: 259.818325
Init 3/200 with method: k-means++
Inertia for init 3/200: 297.517062
Init 4/200 with method: k-means++


C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)
C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


Inertia for init 4/200: 305.223192
Init 5/200 with method: k-means++
Inertia for init 5/200: 291.178815
Init 6/200 with method: k-means++


...

Minibatch iteration 2055/147000: mean batch inertia: 3.867632, ewa inertia: 3.176508 
Minibatch iteration 2056/147000: mean batch inertia: 3.363145, ewa inertia: 3.176762 
Minibatch iteration 2057/147000: mean batch inertia: 3.060719, ewa inertia: 3.176604 
Minibatch iteration 2058/147000: mean batch inertia: 3.320344, ewa inertia: 3.176800 
Minibatch iteration 2059/147000: mean batch inertia: 3.730817, ewa inertia: 3.177554 
Minibatch iteration 2060/147000: mean batch inertia: 3.215929, ewa inertia: 3.177606 
Minibatch iteration 2061/147000: mean batch inertia: 2.704395, ewa inertia: 3.176962 
Minibatch iteration 2062/147000: mean batch inertia: 3.372686, ewa inertia: 3.177228 
Minibatch iteration 2063/147000: mean batch inertia: 3.118159, ewa inertia: 3.177148 
Minibatch iteration 2064/147000: mean batch inertia: 3.608147, ewa inertia: 3.177734 
Minibatch iteration 2065/147000: mean batch inertia: 3.993494, ewa inertia: 3.178845 
Minibatch iteration 2066/147000: mean batch inertia: 2

Minibatch iteration 2159/147000: mean batch inertia: 2.634706, ewa inertia: 3.153668 
Minibatch iteration 2160/147000: mean batch inertia: 3.030688, ewa inertia: 3.153501 
Minibatch iteration 2161/147000: mean batch inertia: 3.143320, ewa inertia: 3.153487 
Minibatch iteration 2162/147000: mean batch inertia: 3.085328, ewa inertia: 3.153394 
Minibatch iteration 2163/147000: mean batch inertia: 3.006101, ewa inertia: 3.153194 
Minibatch iteration 2164/147000: mean batch inertia: 3.169435, ewa inertia: 3.153216 
Minibatch iteration 2165/147000: mean batch inertia: 2.693348, ewa inertia: 3.152590 
Minibatch iteration 2166/147000: mean batch inertia: 2.437325, ewa inertia: 3.151616 
Minibatch iteration 2167/147000: mean batch inertia: 2.853130, ewa inertia: 3.151210 
Minibatch iteration 2168/147000: mean batch inertia: 2.539404, ewa inertia: 3.150377 
Minibatch iteration 2169/147000: mean batch inertia: 3.459741, ewa inertia: 3.150798 
Minibatch iteration 2170/147000: mean batch inertia: 3

Minibatch iteration 2259/147000: mean batch inertia: 3.480058, ewa inertia: 3.135494 
Minibatch iteration 2260/147000: mean batch inertia: 3.148023, ewa inertia: 3.135511 
Minibatch iteration 2261/147000: mean batch inertia: 2.196504, ewa inertia: 3.134234 
Minibatch iteration 2262/147000: mean batch inertia: 3.071345, ewa inertia: 3.134148 
Minibatch iteration 2263/147000: mean batch inertia: 3.634564, ewa inertia: 3.134829 
Minibatch iteration 2264/147000: mean batch inertia: 3.186992, ewa inertia: 3.134900 
Minibatch iteration 2265/147000: mean batch inertia: 2.696022, ewa inertia: 3.134303 
Minibatch iteration 2266/147000: mean batch inertia: 2.844206, ewa inertia: 3.133908 
Minibatch iteration 2267/147000: mean batch inertia: 3.583103, ewa inertia: 3.134519 
Minibatch iteration 2268/147000: mean batch inertia: 3.485726, ewa inertia: 3.134997 
Minibatch iteration 2269/147000: mean batch inertia: 3.654132, ewa inertia: 3.135704 
Minibatch iteration 2270/147000: mean batch inertia: 2

Minibatch iteration 2377/147000: mean batch inertia: 3.679228, ewa inertia: 3.114171 
Minibatch iteration 2378/147000: mean batch inertia: 3.085477, ewa inertia: 3.114132 
Minibatch iteration 2379/147000: mean batch inertia: 2.404858, ewa inertia: 3.113166 
Minibatch iteration 2380/147000: mean batch inertia: 3.289453, ewa inertia: 3.113406 
Minibatch iteration 2381/147000: mean batch inertia: 2.942099, ewa inertia: 3.113173 
Minibatch iteration 2382/147000: mean batch inertia: 2.862656, ewa inertia: 3.112832 
Minibatch iteration 2383/147000: mean batch inertia: 2.913097, ewa inertia: 3.112560 
Minibatch iteration 2384/147000: mean batch inertia: 2.975628, ewa inertia: 3.112374 
Minibatch iteration 2385/147000: mean batch inertia: 3.237925, ewa inertia: 3.112545 
Minibatch iteration 2386/147000: mean batch inertia: 3.007866, ewa inertia: 3.112402 
Minibatch iteration 2387/147000: mean batch inertia: 2.144138, ewa inertia: 3.111085 
Minibatch iteration 2388/147000: mean batch inertia: 3

Minibatch iteration 2492/147000: mean batch inertia: 3.440333, ewa inertia: 3.105165 
Minibatch iteration 2493/147000: mean batch inertia: 2.957123, ewa inertia: 3.104964 
Minibatch iteration 2494/147000: mean batch inertia: 3.170152, ewa inertia: 3.105053 
Minibatch iteration 2495/147000: mean batch inertia: 2.988596, ewa inertia: 3.104894 
Minibatch iteration 2496/147000: mean batch inertia: 2.546939, ewa inertia: 3.104135 
Minibatch iteration 2497/147000: mean batch inertia: 3.714872, ewa inertia: 3.104966 
Minibatch iteration 2498/147000: mean batch inertia: 3.939009, ewa inertia: 3.106101 
Minibatch iteration 2499/147000: mean batch inertia: 2.754140, ewa inertia: 3.105622 
Minibatch iteration 2500/147000: mean batch inertia: 2.413103, ewa inertia: 3.104680 
Minibatch iteration 2501/147000: mean batch inertia: 3.585444, ewa inertia: 3.105334 
Converged (lack of improvement in inertia) at iteration 2501/147000
Computing label assignment and total inertia


In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 400:
        kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 400), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Character Trajectories Data Set 
(http://archive.ics.uci.edu/ml/datasets/Character+Trajectories)

Data Set Information:

The characters here were used for a PhD study on primitive extraction using HMM based models. The data consists of 2858 character samples, contained in the cell array 'mixout'. The struct variable 'consts' contains a field consts.charlabels which provides ennummerated labels for the characters. consts.key provides the key for each label. The data was captured using a WACOM tablet. 3 Dimensions were kept - x, y, and pen tip force. The data has been numerically differentiated and Gaussian smoothed, with a sigma value of 2. Data was captured at 200Hz. The data was normalised with consts.datanorm. Only characters with a single 'PEN-DOWN' segment were considered. Character segmentation was performed using a pen tip force cut-off point. The characters have also been shifted so that their velocity profiles best match the mean of the set.

Attribute Information:

Each character sample is a 3-dimensional pen tip velocity trajectory. This is contained in matrix format, with 3 rows and T columns where T is the length of the character sample.

In [3]:
char = np.load(r"E:\multivariate_time_series_dataset\numpy\CHAR.npz")
X_train = np.empty(shape=(300, ), dtype=object)
y_train = np.empty(shape=(300, ), dtype=object)
X_test = np.empty(shape=(2558, ), dtype=object)
y_test = np.empty(shape=(2558, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(char['X'], char['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(char['Xte'], char['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(char['X'], char['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(char['Xte'], char['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_char.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_char.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [4]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 50), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_char_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_char_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 36.412375
Init 2/200 with method: k-means++
Inertia for init 2/200: 38.957241
Init 3/200 with method: k-means++
Inertia for init 3/200: 40.881089
Init 4/200 with method: k-means++
Inertia for init 4/200: 37.818107
Init 5/200 with method: k-means++
Inertia for init 5/200: 42.685401
Init 6/200 with method: k-means++
Inertia for init 6/200: 38.231006
Init 7/200 with method: k-means++
Inertia for init 7/200: 36.605706
Init 8/200 with method: k-means++
Inertia for init 8/200: 35.314030
Init 9/200 with method: k-means++
Inertia for init 9/200: 38.731324
Init 10/200 with method: k-means++
Inertia for init 10/200: 41.749969
Init 11/200 with method: k-means++
Inertia for init 11/200: 43.197657
Init 12/200 with method: k-means++
Inertia for init 12/200: 37.739229
Init 13/200 with method: k-means++
Inertia for init 13/200: 38.977729
Init 14/200 with method: k-means++
Inertia for init 14/200: 41.048150
Init 15/200 with method: k-means++
Ine

Inertia for init 119/200: 35.880397
Init 120/200 with method: k-means++
Inertia for init 120/200: 41.359890
Init 121/200 with method: k-means++
Inertia for init 121/200: 39.465034
Init 122/200 with method: k-means++
Inertia for init 122/200: 39.614490
Init 123/200 with method: k-means++
Inertia for init 123/200: 37.822621
Init 124/200 with method: k-means++
Inertia for init 124/200: 42.751979
Init 125/200 with method: k-means++
Inertia for init 125/200: 38.022697
Init 126/200 with method: k-means++
Inertia for init 126/200: 39.394675
Init 127/200 with method: k-means++
Inertia for init 127/200: 34.847364
Init 128/200 with method: k-means++
Inertia for init 128/200: 37.243961
Init 129/200 with method: k-means++
Inertia for init 129/200: 38.866651
Init 130/200 with method: k-means++
Inertia for init 130/200: 37.399464
Init 131/200 with method: k-means++
Inertia for init 131/200: 41.934446
Init 132/200 with method: k-means++
Inertia for init 132/200: 39.100990
Init 133/200 with method: k-

Minibatch iteration 78/343000: mean batch inertia: 0.136151, ewa inertia: 0.148553 
Minibatch iteration 79/343000: mean batch inertia: 0.121956, ewa inertia: 0.148538 
Minibatch iteration 80/343000: mean batch inertia: 0.114395, ewa inertia: 0.148518 
Minibatch iteration 81/343000: mean batch inertia: 0.164372, ewa inertia: 0.148527 
Minibatch iteration 82/343000: mean batch inertia: 0.138832, ewa inertia: 0.148522 
Minibatch iteration 83/343000: mean batch inertia: 0.149159, ewa inertia: 0.148522 
Minibatch iteration 84/343000: mean batch inertia: 0.159374, ewa inertia: 0.148528 
Minibatch iteration 85/343000: mean batch inertia: 0.144346, ewa inertia: 0.148526 
Minibatch iteration 86/343000: mean batch inertia: 0.146564, ewa inertia: 0.148525 
Minibatch iteration 87/343000: mean batch inertia: 0.119916, ewa inertia: 0.148508 
Minibatch iteration 88/343000: mean batch inertia: 0.124619, ewa inertia: 0.148494 
Minibatch iteration 89/343000: mean batch inertia: 0.137743, ewa inertia: 0.

Minibatch iteration 207/343000: mean batch inertia: 0.139090, ewa inertia: 0.147922 
Minibatch iteration 208/343000: mean batch inertia: 0.171204, ewa inertia: 0.147936 
Minibatch iteration 209/343000: mean batch inertia: 0.141699, ewa inertia: 0.147932 
Minibatch iteration 210/343000: mean batch inertia: 0.136652, ewa inertia: 0.147926 
Minibatch iteration 211/343000: mean batch inertia: 0.141357, ewa inertia: 0.147922 
Minibatch iteration 212/343000: mean batch inertia: 0.136361, ewa inertia: 0.147915 
Minibatch iteration 213/343000: mean batch inertia: 0.167982, ewa inertia: 0.147927 
Minibatch iteration 214/343000: mean batch inertia: 0.163460, ewa inertia: 0.147936 
Minibatch iteration 215/343000: mean batch inertia: 0.166437, ewa inertia: 0.147947 
Minibatch iteration 216/343000: mean batch inertia: 0.173761, ewa inertia: 0.147962 
Minibatch iteration 217/343000: mean batch inertia: 0.164364, ewa inertia: 0.147971 
Minibatch iteration 218/343000: mean batch inertia: 0.135385, ewa

Minibatch iteration 315/343000: mean batch inertia: 0.184245, ewa inertia: 0.147539 
Minibatch iteration 316/343000: mean batch inertia: 0.139740, ewa inertia: 0.147534 
Minibatch iteration 317/343000: mean batch inertia: 0.124794, ewa inertia: 0.147521 
Minibatch iteration 318/343000: mean batch inertia: 0.147959, ewa inertia: 0.147521 
Minibatch iteration 319/343000: mean batch inertia: 0.137489, ewa inertia: 0.147516 
Minibatch iteration 320/343000: mean batch inertia: 0.106469, ewa inertia: 0.147492 
Minibatch iteration 321/343000: mean batch inertia: 0.142369, ewa inertia: 0.147489 
Minibatch iteration 322/343000: mean batch inertia: 0.130707, ewa inertia: 0.147479 
Minibatch iteration 323/343000: mean batch inertia: 0.131856, ewa inertia: 0.147470 
Minibatch iteration 324/343000: mean batch inertia: 0.152855, ewa inertia: 0.147473 
Minibatch iteration 325/343000: mean batch inertia: 0.149142, ewa inertia: 0.147474 
Minibatch iteration 326/343000: mean batch inertia: 0.166873, ewa

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  7.1min finished


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  1.6min finished


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  7.4min finished


In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 50:
        kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 50), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   ChlorineConcentration: Chlorine concentration data set 
(https://rdrr.io/github/moviedo5/fda.tsc/man/ChlorineConcentration.html)

Data Set Information:

The data set consists of 166 nodes (pipe junctions) and measurement of the Chlorine concentration level at all these nodes during 15 days (one measurement for every 5 minutes, a total of 4310 time ticks). 

This dataset was defined in a PhD thesis by Lei Li (Carnegie Mellon University). It was produced by EPANET that models the hydraulic and water quality behavior of water distribution piping systems. EPANET can track, in a given water network, the water level and pressure in each tank, the water flow in the pipes and the concentration of a chemical species (Chlorine in this case) throughout the network within a simulated duration. 

Attribute Information:



- class: Corresponding class level of “ChlorineConcentration” curves with 3 classes with 1000, 1000 and 2307 observations per class respectively.

- sample:Factor variable. In TSC database, the first 467 values (sample=train) are used for training sample and the rest of 3840 (sample=test) for testing.

In [ ]:
chlo = np.load(r"E:\multivariate_time_series_dataset\numpy\CHLO.npz")
X_train = np.empty(shape=(467, ), dtype=object)
y_train = np.empty(shape=(467, ), dtype=object)
X_test = np.empty(shape=(3840, ), dtype=object)
y_test = np.empty(shape=(3840, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(chlo['X'], chlo['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(chlo['Xte'], chlo['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(chlo['X'], chlo['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(chlo['Xte'], chlo['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_chlo.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_chlo.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_chlo_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_chlo_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
plt.plot(np.concatenate(X_train[:2]))

##  CMU Graphics Lab Motion Capture Database
(http://mocap.cs.cmu.edu/)

Data Set Information:

The mocap lab in the basement of Wean contains 12 Vicon infrared MX-40 cameras, each of which is capable of recording 120 Hz with images of 4 megapixel resolution. The cameras are placed around a rectangular area, of approximately 3m x 8m, in the center of the room. Only motions that take place in this rectangle can be captured. If motion of human hands is being captured, more detail is required and the cameras are moved closer to capture a smaller space with higher resolution.

To capture something, small grey markers are placed on it. Humans wear a black jumpsuit and have 41 markers taped on. The Vicon cameras see the markers in infra-red. The images that the various cameras pick up are triangulated to get 3D data.

Attribute Information:

 ViconIQ requires user interaction to start off the skeleton fitting. To process a capture, a segment of motion is loaded onscreen as a point cloud of markers. The user goes through and specifies the correspondence between these markers and the markers in the .vst, e.g. "this white dot is the clavicle marker". From this data ViconIQ can fit a skeleton and determine the skeleton's limb lengths. From here on out the labeling process is automatic. ViconIQ can load up each motion clip and automatically perform a "Kinematic Fit" of the skeleton to the markers. During this time the software uses its knowledge of the skeleton to correct captured marker aberrations. The user can also fix things up by editing the joint rotation/translation graphs directly. 

In [ ]:
cmu = np.load(r"E:\multivariate_time_series_dataset\numpy\CMU.npz")
X_train = np.empty(shape=(29, ), dtype=object)
y_train = np.empty(shape=(29, ), dtype=object)
X_test = np.empty(shape=(29, ), dtype=object)
y_test = np.empty(shape=(29, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(cmu['X'], cmu['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(cmu['Xte'], cmu['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(cmu['X'], cmu['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(cmu['Xte'], cmu['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_cmu.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_cmu.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_cmu_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_cmu_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  ECG dataset

Data Set Information:

TODO

Attribute Information:

TODO

In [ ]:
ecg = np.load(r"E:\multivariate_time_series_dataset\numpy\ECG.npz")
X_train = np.empty(shape=(100, ), dtype=object)
y_train = np.empty(shape=(100, ), dtype=object)
X_test = np.empty(shape=(100, ), dtype=object)
y_test = np.empty(shape=(100, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(ecg['X'], ecg['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(ecg['Xte'], ecg['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(ecg['X'], ecg['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(ecg['Xte'], ecg['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_ecg.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_ecg.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_ecg_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_ecg_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   Japanese Vowels Data Set 

Data Set Information:

The data was collected for examining our newly developed classifier for multidimensional curves (multidimensional time series). Nine male speakers uttered two Japanese vowels /ae/ successively. For each utterance, with the analysis parameters described below, we applied 12-degree linear prediction analysis to it to obtain a discrete-time series with 12 LPC cepstrum coefficients. This means that one utterance by a speaker forms a time series whose length is in the range 7-29 and each point of a time series is of 12 features (12 coefficients).

The number of the time series is 640 in total. We used one set of 270 time series for training and the other set of 370 time series for testing.

- Number of Instances (Utterances):
    - Training: 270 (30 utterances by 9 speakers. See file 'size_ae.train'.)
    - Testing: 370 (24-88 utterances by the same 9 speakers in different opportunities. See file 'size_ae.test'.)

- Length of Time Series:
     - 7 - 29 depending on utterances

- Analysis parameters:
    - Sampling rate : 10kHz
    - Frame length : 25.6 ms
    - Shift length : 6.4ms
    - Degree of LPC coefficients : 12 

Attribute Information:

12 Real Attributes

In [5]:
jpvow = np.load(r"E:\multivariate_time_series_dataset\numpy\JPVOW.npz")
X_train = np.empty(shape=(270, ), dtype=object)
y_train = np.empty(shape=(270, ), dtype=object)
X_test = np.empty(shape=(370, ), dtype=object)
y_test = np.empty(shape=(370, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(jpvow['X'], jpvow['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(jpvow['Xte'], jpvow['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(jpvow['X'], jpvow['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(jpvow['Xte'], jpvow['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_jpvow.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_jpvow.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [6]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 50), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_jpvow_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_jpvow_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 860.398663
Init 2/200 with method: k-means++
Inertia for init 2/200: 894.565307
Init 3/200 with method: k-means++
Inertia for init 3/200: 858.110492
Init 4/200 with method: k-means++
Inertia for init 4/200: 881.408851
Init 5/200 with method: k-means++
Inertia for init 5/200: 901.060799
Init 6/200 with method: k-means++
Inertia for init 6/200: 884.335380
Init 7/200 with method: k-means++
Inertia for init 7/200: 843.834349
Init 8/200 with method: k-means++
Inertia for init 8/200: 865.322659
Init 9/200 with method: k-means++
Inertia for init 9/200: 870.379731
Init 10/200 with method: k-means++
Inertia for init 10/200: 851.653824
Init 11/200 with method: k-means++
Inertia for init 11/200: 873.335226
Init 12/200 with method: k-means++
Inertia for init 12/200: 953.702927
Init 13/200 with method: k-means++
Inertia for init 13/200: 835.524590
Init 14/200 with method: k-means++
Inertia for init 14/200: 877.564724
Init 15/200 with method:

Inertia for init 119/200: 820.345957
Init 120/200 with method: k-means++
Inertia for init 120/200: 851.579517
Init 121/200 with method: k-means++
Inertia for init 121/200: 888.331787
Init 122/200 with method: k-means++
Inertia for init 122/200: 867.889879
Init 123/200 with method: k-means++
Inertia for init 123/200: 872.307836
Init 124/200 with method: k-means++
Inertia for init 124/200: 914.978777
Init 125/200 with method: k-means++
Inertia for init 125/200: 841.886001
Init 126/200 with method: k-means++
Inertia for init 126/200: 850.907370
Init 127/200 with method: k-means++
Inertia for init 127/200: 901.474166
Init 128/200 with method: k-means++
Inertia for init 128/200: 916.032081
Init 129/200 with method: k-means++
Inertia for init 129/200: 850.171102
Init 130/200 with method: k-means++
Inertia for init 130/200: 903.317707
Init 131/200 with method: k-means++
Inertia for init 131/200: 867.555077
Init 132/200 with method: k-means++
Inertia for init 132/200: 886.826780
Init 133/200 w

Minibatch iteration 75/10000: mean batch inertia: 3.366456, ewa inertia: 3.227180 
Minibatch iteration 76/10000: mean batch inertia: 3.307039, ewa inertia: 3.228783 
Minibatch iteration 77/10000: mean batch inertia: 3.061087, ewa inertia: 3.225417 
Minibatch iteration 78/10000: mean batch inertia: 2.808983, ewa inertia: 3.217056 
Minibatch iteration 79/10000: mean batch inertia: 2.994955, ewa inertia: 3.212597 
Minibatch iteration 80/10000: mean batch inertia: 3.478489, ewa inertia: 3.217935 
Minibatch iteration 81/10000: mean batch inertia: 2.932988, ewa inertia: 3.212215 
Minibatch iteration 82/10000: mean batch inertia: 3.123548, ewa inertia: 3.210434 
Minibatch iteration 83/10000: mean batch inertia: 2.971507, ewa inertia: 3.205638 
Minibatch iteration 84/10000: mean batch inertia: 3.158367, ewa inertia: 3.204689 
Minibatch iteration 85/10000: mean batch inertia: 3.099183, ewa inertia: 3.202571 
Minibatch iteration 86/10000: mean batch inertia: 3.070382, ewa inertia: 3.199917 
Mini

Minibatch iteration 195/10000: mean batch inertia: 3.319107, ewa inertia: 3.157721 
Minibatch iteration 196/10000: mean batch inertia: 3.050995, ewa inertia: 3.155579 
Minibatch iteration 197/10000: mean batch inertia: 3.130693, ewa inertia: 3.155079 
Minibatch iteration 198/10000: mean batch inertia: 3.020479, ewa inertia: 3.152377 
Minibatch iteration 199/10000: mean batch inertia: 3.192302, ewa inertia: 3.153178 
Minibatch iteration 200/10000: mean batch inertia: 3.178430, ewa inertia: 3.153685 
Minibatch iteration 201/10000: mean batch inertia: 3.447538, ewa inertia: 3.159585 
Minibatch iteration 202/10000: mean batch inertia: 3.365375, ewa inertia: 3.163716 
Minibatch iteration 203/10000: mean batch inertia: 2.889752, ewa inertia: 3.158216 
Minibatch iteration 204/10000: mean batch inertia: 3.095601, ewa inertia: 3.156959 
Minibatch iteration 205/10000: mean batch inertia: 3.014193, ewa inertia: 3.154093 
Minibatch iteration 206/10000: mean batch inertia: 3.180212, ewa inertia: 3.

Minibatch iteration 321/10000: mean batch inertia: 2.949595, ewa inertia: 3.140305 
Minibatch iteration 322/10000: mean batch inertia: 3.074342, ewa inertia: 3.138981 
Minibatch iteration 323/10000: mean batch inertia: 3.136428, ewa inertia: 3.138930 
Minibatch iteration 324/10000: mean batch inertia: 3.120585, ewa inertia: 3.138562 
Minibatch iteration 325/10000: mean batch inertia: 3.205251, ewa inertia: 3.139900 
Minibatch iteration 326/10000: mean batch inertia: 3.322499, ewa inertia: 3.143566 
Minibatch iteration 327/10000: mean batch inertia: 2.987179, ewa inertia: 3.140427 
Minibatch iteration 328/10000: mean batch inertia: 3.174245, ewa inertia: 3.141106 
Minibatch iteration 329/10000: mean batch inertia: 3.138021, ewa inertia: 3.141044 
Minibatch iteration 330/10000: mean batch inertia: 3.349743, ewa inertia: 3.145234 
Minibatch iteration 331/10000: mean batch inertia: 3.615871, ewa inertia: 3.154682 
Minibatch iteration 332/10000: mean batch inertia: 2.931083, ewa inertia: 3.

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.1min finished


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:   54.9s finished


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  4.1min finished


In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 50:
        kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 50), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##   Kicks vs. Punch Dataset
(https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7813879/)

Data Set Information:

TODO

Attribute Information:

TODO

In [7]:
kick = np.load(r"E:\multivariate_time_series_dataset\numpy\KICK.npz")
X_train = np.empty(shape=(16, ), dtype=object)
y_train = np.empty(shape=(16, ), dtype=object)
X_test = np.empty(shape=(10, ), dtype=object)
y_test = np.empty(shape=(10, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(kick['X'], kick['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(kick['Xte'], kick['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(kick['X'], kick['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(kick['Xte'], kick['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_kick.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_kick.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [9]:
kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 100), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_kick_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_kick_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 3190.035686
Init 2/200 with method: k-means++
Inertia for init 2/200: 2897.591869
Init 3/200 with method: k-means++
Inertia for init 3/200: 3140.507830
Init 4/200 with method: k-means++
Inertia for init 4/200: 3131.750272
Init 5/200 with method: k-means++
Inertia for init 5/200: 3147.290125
Init 6/200 with method: k-means++
Inertia for init 6/200: 2954.979400
Init 7/200 with method: k-means++
Inertia for init 7/200: 3162.499527
Init 8/200 with method: k-means++
Inertia for init 8/200: 3195.411027
Init 9/200 with method: k-means++
Inertia for init 9/200: 3142.987939
Init 10/200 with method: k-means++
Inertia for init 10/200: 2832.706655
Init 11/200 with method: k-means++
Inertia for init 11/200: 3134.010328
Init 12/200 with method: k-means++
Inertia for init 12/200: 3244.864073
Init 13/200 with method: k-means++
Inertia for init 13/200: 3264.206035
Init 14/200 with method: k-means++
Inertia for init 14/200: 3140.414020
Init 15/20

Inertia for init 120/200: 3072.840758
Init 121/200 with method: k-means++
Inertia for init 121/200: 3408.095228
Init 122/200 with method: k-means++
Inertia for init 122/200: 3221.030872
Init 123/200 with method: k-means++
Inertia for init 123/200: 3248.501583
Init 124/200 with method: k-means++
Inertia for init 124/200: 2943.776979
Init 125/200 with method: k-means++
Inertia for init 125/200: 3029.565691
Init 126/200 with method: k-means++
Inertia for init 126/200: 2668.838247
Init 127/200 with method: k-means++
Inertia for init 127/200: 3078.536644
Init 128/200 with method: k-means++
Inertia for init 128/200: 3058.100847
Init 129/200 with method: k-means++
Inertia for init 129/200: 3245.700994
Init 130/200 with method: k-means++
Inertia for init 130/200: 2910.889365
Init 131/200 with method: k-means++
Inertia for init 131/200: 2890.923917
Init 132/200 with method: k-means++
Inertia for init 132/200: 3007.067895
Init 133/200 with method: k-means++
Inertia for init 133/200: 3234.454716


Minibatch iteration 90/11100: mean batch inertia: 13.242025, ewa inertia: 13.621701 
Minibatch iteration 91/11100: mean batch inertia: 12.082249, ewa inertia: 13.593953 
Minibatch iteration 92/11100: mean batch inertia: 12.619724, ewa inertia: 13.576393 
Minibatch iteration 93/11100: mean batch inertia: 11.897750, ewa inertia: 13.546136 
Minibatch iteration 94/11100: mean batch inertia: 11.737276, ewa inertia: 13.513533 
Minibatch iteration 95/11100: mean batch inertia: 13.198302, ewa inertia: 13.507851 
Minibatch iteration 96/11100: mean batch inertia: 13.220015, ewa inertia: 13.502663 
Minibatch iteration 97/11100: mean batch inertia: 12.407492, ewa inertia: 13.482923 
Minibatch iteration 98/11100: mean batch inertia: 12.771550, ewa inertia: 13.470101 
Minibatch iteration 99/11100: mean batch inertia: 12.183481, ewa inertia: 13.446910 
Minibatch iteration 100/11100: mean batch inertia: 13.691200, ewa inertia: 13.451313 
Minibatch iteration 101/11100: mean batch inertia: 11.589367, ew

Minibatch iteration 247/11100: mean batch inertia: 12.119299, ewa inertia: 12.277944 
Minibatch iteration 248/11100: mean batch inertia: 13.723644, ewa inertia: 12.304002 
Minibatch iteration 249/11100: mean batch inertia: 11.770500, ewa inertia: 12.294386 
Minibatch iteration 250/11100: mean batch inertia: 12.040336, ewa inertia: 12.289807 
Minibatch iteration 251/11100: mean batch inertia: 11.794632, ewa inertia: 12.280882 
Minibatch iteration 252/11100: mean batch inertia: 11.599440, ewa inertia: 12.268599 
Minibatch iteration 253/11100: mean batch inertia: 12.028069, ewa inertia: 12.264264 
Minibatch iteration 254/11100: mean batch inertia: 11.722001, ewa inertia: 12.254489 
Minibatch iteration 255/11100: mean batch inertia: 11.837435, ewa inertia: 12.246972 
Minibatch iteration 256/11100: mean batch inertia: 11.816186, ewa inertia: 12.239208 
Minibatch iteration 257/11100: mean batch inertia: 12.918125, ewa inertia: 12.251445 
Minibatch iteration 258/11100: mean batch inertia: 11.

Minibatch iteration 402/11100: mean batch inertia: 14.297543, ewa inertia: 12.035487 
Minibatch iteration 403/11100: mean batch inertia: 12.675439, ewa inertia: 12.047021 
Minibatch iteration 404/11100: mean batch inertia: 11.321561, ewa inertia: 12.033945 
Minibatch iteration 405/11100: mean batch inertia: 12.915566, ewa inertia: 12.049836 
Minibatch iteration 406/11100: mean batch inertia: 12.213516, ewa inertia: 12.052786 
Minibatch iteration 407/11100: mean batch inertia: 12.449913, ewa inertia: 12.059944 
Minibatch iteration 408/11100: mean batch inertia: 12.539002, ewa inertia: 12.068579 
Minibatch iteration 409/11100: mean batch inertia: 12.114381, ewa inertia: 12.069405 
Minibatch iteration 410/11100: mean batch inertia: 12.297153, ewa inertia: 12.073510 
Minibatch iteration 411/11100: mean batch inertia: 11.816078, ewa inertia: 12.068870 
Minibatch iteration 412/11100: mean batch inertia: 11.643104, ewa inertia: 12.061195 
Minibatch iteration 413/11100: mean batch inertia: 12.

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.6s finished


Fitting 2 folds for each of 11 candidates, totalling 22 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    6.6s finished


Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   30.2s finished


In [ ]:
from sklearn.decomposition import PCA


pca_dataset = PCA(whiten=True)
pca_dataset.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
X_decomposed = pca_dataset.transform(np.concatenate(np.concatenate((X_train, X_test))))

fig, ax11 = plt.subplots()
ax11.bar(range(1, np.concatenate(np.concatenate((X_train, X_test))).shape[1]+1), 100*pca_dataset.explained_variance_ratio_)
ax11.set_xlabel("Number of Components")
ax11.set_ylabel("Explained Variance in \%")

ax12 = ax11.twinx()
ax12.plot(range(1, np.concatenate(np.concatenate((X_train, X_test))).shape[1]+1), 100*np.cumsum(pca_dataset.explained_variance_ratio_), '-ro', alpha=0.5)
ax12.axhline(y=95, ls='-', c='g')
ax12.set_ylabel("Accumulated Variance in \%")

width = 3.487
height =width / 1.618
fig.set_size_inches(width, height)
fig.subplots_adjust(wspace=1.5)
# plt.savefig('ParetoChart_cleaned.pdf', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
plt.figure()
if X_decomposed.shape[1] > 2:
    ax = plt.axes(projection='3d')
    plt.scatter(X_decomposed[:, 0], X_decomposed[:, 1], X_decomposed[:, 2], cmap=plt.cm.RdBu, rasterized=True, c=kmeans.labels_)
    ax.set_zlabel("Component 3")
else:
    ax = plt.axes()
    plt.scatter(X_decomposed[:, 0], X_decomposed[:, 1], cmap=plt.cm.RdBu, rasterized=True, c=kmeans.labels_)
ax.set_xlabel("Component 1")
ax.set_ylabel("Component 2")
plt.tight_layout()

width = 3.487
height =width / 1.618
fig.set_size_inches(width, height)
fig.subplots_adjust(wspace=1.5)
# plt.savefig('PCA_cleaned.pdf', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 100:
        kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 100), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##    Libras Movement Data Set
(https://archive.ics.uci.edu/ml/datasets/Libras+Movement)

Data Set Information:

The dataset (movement_libras) contains 15 classes of 24 instances each, where each class references to a hand movement type in LIBRAS.

In the video pre-processing, a time normalization is carried out selecting 45 frames from each video, in according
to an uniform distribution. In each frame, the centroid pixels of the segmented objects (the hand) are found, which
compose the discrete version of the curve F with 45 points. All curves are normalized in the unitary space.

In order to prepare these movements to be analysed by algorithms, we have carried out a mapping operation, that is, each
curve F is mapped in a representation with 90 features, with representing the coordinates of movement.

Some sub-datasets are offered in order to support comparisons of results. 

Attribute Information:

90 numeric (double) and 1 for the class (integer)

In [14]:
lib = np.load(r"E:\multivariate_time_series_dataset\numpy\LIB.npz")
X_train = np.empty(shape=(180, ), dtype=object)
y_train = np.empty(shape=(180, ), dtype=object)
X_test = np.empty(shape=(180, ), dtype=object)
y_test = np.empty(shape=(180, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(lib['X'], lib['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(lib['Xte'], lib['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(lib['X'], lib['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(lib['Xte'], lib['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_lib.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_lib.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN (TODO: RENAME and SPARSE)

In [18]:
kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 1600 - 400), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 1600,
                          'k_in': 2,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_lib_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_lib_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 0.681000
Init 2/200 with method: k-means++


C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)
C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


Inertia for init 2/200: 0.444726
Init 3/200 with method: k-means++
Inertia for init 3/200: 0.681551
Init 4/200 with method: k-means++


C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)
C:\Users\Steiner\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1676: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


Inertia for init 4/200: 0.499404
Init 5/200 with method: k-means++
Inertia for init 5/200: 0.708743
Init 6/200 with method: k-means++


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 12.9min finished


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  2.8min finished


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 11.9min finished


In [ ]:
fig = plt.figure()
ax = plt.axes()
plt.scatter(np.concatenate(X_train)[:, 0], np.concatenate(X_train)[:, 1], cmap=plt.cm.RdBu, rasterized=True, c=kmeans.labels_)
ax.set_xlabel("Feature 1")
ax.set_ylabel("Feature 2")
plt.tight_layout()

width = 3.487
height =width / 1.618
fig.set_size_inches(width, height)
fig.subplots_adjust(wspace=1.5)
plt.savefig('lib_k_400.pdf', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 400:
        kmeans = MiniBatchKMeans(n_clusters=400, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 400), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##    NetFlow dataset


Data Set Information:

TODO

Attribute Information:

TODO

TODO: Shuffle, find a better random state

In [19]:
net = np.load(r"E:\multivariate_time_series_dataset\numpy\NET.npz")
X_train = np.empty(shape=(803, ), dtype=object)
y_train = np.empty(shape=(803, ), dtype=object)
X_test = np.empty(shape=(534, ), dtype=object)
y_test = np.empty(shape=(534, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(net['X'], net['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(net['Xte'], net['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(net['X'], net['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(net['Xte'], net['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 4,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_net.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_net.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [20]:
kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 100), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 4,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_net_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_net_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 0.494128
Init 2/200 with method: k-means++
Inertia for init 2/200: 0.263712
Init 3/200 with method: k-means++
Inertia for init 3/200: 0.063756
Init 4/200 with method: k-means++
Inertia for init 4/200: 0.341841
Init 5/200 with method: k-means++
Inertia for init 5/200: 0.429972
Init 6/200 with method: k-means++
Inertia for init 6/200: 1.064744
Init 7/200 with method: k-means++
Inertia for init 7/200: 0.224021
Init 8/200 with method: k-means++
Inertia for init 8/200: 0.502849
Init 9/200 with method: k-means++
Inertia for init 9/200: 0.167693
Init 10/200 with method: k-means++
Inertia for init 10/200: 0.181575
Init 11/200 with method: k-means++
Inertia for init 11/200: 0.124060
Init 12/200 with method: k-means++
Inertia for init 12/200: 0.243499
Init 13/200 with method: k-means++
Inertia for init 13/200: 0.488753
Init 14/200 with method: k-means++
Inertia for init 14/200: 0.370937
Init 15/200 with method: k-means++
Inertia for init 

Inertia for init 119/200: 0.318665
Init 120/200 with method: k-means++
Inertia for init 120/200: 0.511351
Init 121/200 with method: k-means++
Inertia for init 121/200: 0.160176
Init 122/200 with method: k-means++
Inertia for init 122/200: 0.303003
Init 123/200 with method: k-means++
Inertia for init 123/200: 0.068385
Init 124/200 with method: k-means++
Inertia for init 124/200: 0.168852
Init 125/200 with method: k-means++
Inertia for init 125/200: 0.172022
Init 126/200 with method: k-means++
Inertia for init 126/200: 0.176695
Init 127/200 with method: k-means++
Inertia for init 127/200: 0.486969
Init 128/200 with method: k-means++
Inertia for init 128/200: 0.607098
Init 129/200 with method: k-means++
Inertia for init 129/200: 0.083457
Init 130/200 with method: k-means++
Inertia for init 130/200: 0.336721
Init 131/200 with method: k-means++
Inertia for init 131/200: 0.657970
Init 132/200 with method: k-means++
Inertia for init 132/200: 0.130800
Init 133/200 with method: k-means++
Inerti

KeyboardInterrupt: 

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 100:
        kmeans = MiniBatchKMeans(n_clusters=100, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 100), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

## PEMS-SF Data Set


Data Set Information:

15 months worth of daily data from the California Department of Transportation PEMS website, (http://pems.dot.ca.gov/), the data describes the occupancy rate, between 0 and 1, of different car lanes of San Francisco bay area freeways. The measurements cover the period from Jan. 1st 2008 to Mar. 30th 2009 and are sampled every 10 minutes. We consider each day in this database as a single time series of dimension 963 (the number of sensors which functioned consistently throughout the studied period) and length 6 x 24=144. We remove public holidays from the dataset, as well as two days with anomalies (March 8th 2009 and March 9th 2008) where all sensors were muted between 2:00 and 3:00 AM. This results in a database of 440 time series.

The task we propose on this dataset is to classify each observed day as the correct day of the week, from Monday to Sunday, e.g. label it with an integer in {1,2,3,4,5,6,7}.

I will keep separate copies of this database on my website in a Matlab format. If you use Matlab, it might be more convenient to consider these .mat files directly.

There are two files for each fold, the data file and the labels file. We have split the 440 time series between train and test folds, but you are of course free to merge them to consider a different cross validation setting.
- The PEMS_train textfile has 263 lines. Each line describes a time-series provided as a matrix. The matrix syntax is that of Matlab, e.g. [ a b ; c d] is the matrix with row vectors [a b] and [c d] in that order. Each matrix describes the different occupancies rates (963 lines, one for each station/detector) sampled every 10 minutes during the day (144 columns).
- The PEMS_trainlabel text describes, for each day of measurements described above, the day of the week on which the data was sampled, namely an integer between 1 (Mon.) and 7 (Sun.).

- PEMS_test and PEMS_testlabels are formatted in the same way, except that there are 173 test instances.

- The permutation that I used to shuffle the dataset is given in the randperm file. If you need to rearrange the data so that it follows the calendar order, you should merge train and test samples and reorder them using the inverse permutation of randperm.

Attribute Information:

Each attribute describes the measurement of the occupancy rate (between 0 and 1) of a captor location as recorded by a measuring station, at a given timestamp in time during the day. The ID of each station is given in the stations_list text file. For more information on the location (GPS, Highway, Direction) of each station please refer to the PEMS website. There are 963 (stations) x 144 (timestamps) = 138.672 attributes for each record.

In [ ]:
pems = np.load(r"E:\multivariate_time_series_dataset\numpy\PEMS.npz")
X_train = np.empty(shape=(267, ), dtype=object)
y_train = np.empty(shape=(267, ), dtype=object)
X_test = np.empty(shape=(173, ), dtype=object)
y_test = np.empty(shape=(173, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(pems['X'], pems['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(pems['Xte'], pems['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(pems['X'], pems['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(pems['Xte'], pems['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_pems.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_pems.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_pems_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_pems_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

## DistalPhalanxTW Data Set


Data Set Information:

This series of 11 classification problems were created as part of Luke Davis's PhD titled "Predictive Modelling of Bone Ageing". They are all derived from the same images, extracted from Cao et al. "Digital hand atlas and web-based bone age assessment: system design and implementation". They are designed to test the efficacy of hand and bone outline detection and whether these outlines could be helpful in bone age prediction. Algorithms to automatically extract the hand outlines and then the outlines of three bones of the middle finger (proximal, middle and distal phalanges) were applied to over 1300 images, and three human evaluators labelled the output of the image outlining as correct or incorrect. This generated three classification problems: DistalPhalanxOutlineCorrect; MiddlePhalanxOutlineCorrect; and ProximalPhalanxOutlineCorrect. The next stage of the project was to use the outlines to predict information about the subjects age. The three problems {\em DistalPhalanxOutlineAgeGroup, MiddlePhalanxOutlineAgeGroup and ProximalPhalanxOutlineAgeGroup} involve using the outline of one of the phalanges to predict whether the subject is one of three age groups: 0-6 years old, 7-12 years old and 13-19 years old. Note that these problems are aligned by subject, and hence can be treated as a multi dimensional TSC problem. Problem Phalanges contains the concatenation of all three problems. Bone age estimation is usually performed by an expert with an algorithm called Tanner-Whitehouse. This involves scoring each bone into one of seven categories based on the stage of development. The final three bone image classification problems, DistalPhalanxTW, MiddlePhalanxTW and ProximalPhalanxTW}, involve predicting the Tanner-Whitehouse score (as labelled by a human expert) from the outline.

Attribute Information:

TODO

In [ ]:
phal = np.load(r"E:\multivariate_time_series_dataset\numpy\PHAL.npz")
X_train = np.empty(shape=(400, ), dtype=object)
y_train = np.empty(shape=(400, ), dtype=object)
X_test = np.empty(shape=(139, ), dtype=object)
y_test = np.empty(shape=(139, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(phal['X'], phal['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(phal['Xte'], phal['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(phal['X'], phal['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(phal['Xte'], phal['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_phal.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_phal.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = MinMaxScaler(feature_range=(-1, 1)).fit_transform(kmeans.cluster_centers_)
# w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_phal_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_phal_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
plt.plot(np.concatenate(X_train[:1]))

##  Robot Execution Failures Data Set 
(http://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures)

Data Set Information:

The donation includes 5 datasets, each of them defining a different learning problem:

- LP1: failures in approach to grasp position
- LP2: failures in transfer of a part
- LP3: position of part after a transfer failure
- LP4: failures in approach to ungrasp position
- LP5: failures in motion with part

In order to improve classification accuracy, a set of five feature transformation strategies (based on statistical summary features, discrete Fourier transform, etc.) was defined and evaluated. This enabled an average improvement of 20% in accuracy. The most accessible reference is Seabra Lopes and Camarinha-Matos, 1998.

Attribute Information:

All features are numeric although they are integer valued only. Each feature represents a force or a torque measured after failure detection; each failure instance is characterized in terms of 15 force/torque samples collected at regular time intervals starting immediately after failure detection; The total observation window for each failure instance was of 315 ms.

Each example is described as follows:

class
Fx1 Fy1 Fz1 Tx1 Ty1 Tz1
Fx2 Fy2 Fz2 Tx2 Ty2 Tz2
......
Fx15 Fy15 Fz15 Tx15 Ty15 Tz15

where Fx1 ... Fx15 is the evolution of force Fx in the observation window, the same for Fy, Fz and the torques; there is a total of 90 features.

In [ ]:
robot = np.load(r"E:\multivariate_time_series_dataset\numpy\ROBOT.npz")
X_train = np.empty(shape=(100, ), dtype=object)
y_train = np.empty(shape=(100, ), dtype=object)
X_test = np.empty(shape=(64, ), dtype=object)
y_test = np.empty(shape=(64, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(robot['X'], robot['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(robot['Xte'], robot['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(robot['X'], robot['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(robot['Xte'], robot['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_robot.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_robot.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_robot_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_robot_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  SwedishLeaf Data Set 
(http://www.timeseriesclassification.com/description.php?Dataset=SwedishLeaf)

Data Set Information:

Swedish leaf is a set of leaf outlines donated by Oskar S�derkvist, and used in his MSc thesis Computer vision classifcation of leaves from swedish trees, 2001. The tree classes are 1. Ulmus carpinifolia 2. Acer 3. Salix aurita 4. Quercus 5. Alnus incana 6. Betula pubescens 7. Salix alba 'Sericea' 8. Populus tremula 9. Ulmus glabra 10. Sorbus aucuparia 11. Salix sinerea 12. Populus 13.Tilia 14. Sorbus intermedia 15. Fagus silvatica 

Attribute Information:

TODO

In [ ]:
swe = np.load(r"E:\multivariate_time_series_dataset\numpy\SWE.npz")
X_train = np.empty(shape=(500, ), dtype=object)
y_train = np.empty(shape=(500, ), dtype=object)
X_test = np.empty(shape=(625, ), dtype=object)
y_test = np.empty(shape=(625, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(swe['X'], swe['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(swe['Xte'], swe['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(swe['X'], swe['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(swe['Xte'], swe['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_swe.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_swe.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = MinMaxScaler(feature_range=(-1, 1)).fit_transform(kmeans.cluster_centers_)

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 1,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_swe_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_swe_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
plt.plot(np.concatenate(X_train[:5]))

##  uWave Data Set 
(https://www.yecl.org/publications/liu09percom.pdf)

Data Set Information:

TODO

Attribute Information:

TODO

In [ ]:
uwav = np.load(r"E:\multivariate_time_series_dataset\numpy\UWAV.npz")
X_train = np.empty(shape=(200, ), dtype=object)
y_train = np.empty(shape=(200, ), dtype=object)
X_test = np.empty(shape=(428, ), dtype=object)
y_test = np.empty(shape=(428, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(uwav['X'], uwav['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(uwav['Xte'], uwav['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(uwav['X'], uwav['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(uwav['Xte'], uwav['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_uwav.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_uwav.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 3,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_uwav_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_uwav_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  Wafer Data Set 
(http://www.timeseriesclassification.com/description.php?Dataset=Wafer)

Data Set Information:

This dataset was formatted by R. Olszewski as part of his thesis Generalized feature extraction for structural pattern recognition in time-series data at Carnegie Mellon University, 2001. Wafer data relates to semi-conductor microelectronics fabrication. A collection of inline process control measurements recorded from various sensors during the processing of silicon wafers for semiconductor fabrication constitute the wafer database; each data set in the wafer database contains the measurements recorded by one sensor during the processing of one wafer by one tool. The two classes are normal and abnormal. There is a large class imbalance between normal and abnormal (10.7% of the train are abnormal, 12.1% of the test). 

Attribute Information:

TODO

In [21]:
waf = np.load(r"E:\multivariate_time_series_dataset\numpy\WAF.npz")
X_train = np.empty(shape=(298, ), dtype=object)
y_train = np.empty(shape=(298, ), dtype=object)
X_test = np.empty(shape=(896, ), dtype=object)
y_test = np.empty(shape=(896, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(waf['X'], waf['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(waf['Xte'], waf['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(waf['X'], waf['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(waf['Xte'], waf['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_waf.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_waf.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN

In [23]:
kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
w_in = np.pad(w_in, ((0, 800 - 200), (0, 0)), mode='constant', constant_values=0)

initially_fixed_params = {'hidden_layer_size': 800,
                          'k_in': 5,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_waf_km_sparse.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_waf_km_sparse.joblib")

Init 1/200 with method: k-means++
Inertia for init 1/200: 5.034372
Init 2/200 with method: k-means++
Inertia for init 2/200: 1.303531
Init 3/200 with method: k-means++
Inertia for init 3/200: 14.386570
Init 4/200 with method: k-means++
Inertia for init 4/200: 20.127613
Init 5/200 with method: k-means++
Inertia for init 5/200: 8.088663
Init 6/200 with method: k-means++
Inertia for init 6/200: 3.523553
Init 7/200 with method: k-means++
Inertia for init 7/200: 2.121652
Init 8/200 with method: k-means++
Inertia for init 8/200: 23.843817
Init 9/200 with method: k-means++
Inertia for init 9/200: 3.124773
Init 10/200 with method: k-means++
Inertia for init 10/200: 8.841286
Init 11/200 with method: k-means++
Inertia for init 11/200: 5.047710
Init 12/200 with method: k-means++
Inertia for init 12/200: 2.859102
Init 13/200 with method: k-means++
Inertia for init 13/200: 2.883923
Init 14/200 with method: k-means++
Inertia for init 14/200: 2.486586
Init 15/200 with method: k-means++
Inertia for in

Inertia for init 119/200: 5.097120
Init 120/200 with method: k-means++
Inertia for init 120/200: 3.078909
Init 121/200 with method: k-means++
Inertia for init 121/200: 16.004988
Init 122/200 with method: k-means++
Inertia for init 122/200: 2.364164
Init 123/200 with method: k-means++
Inertia for init 123/200: 3.560895
Init 124/200 with method: k-means++
Inertia for init 124/200: 15.636125
Init 125/200 with method: k-means++
Inertia for init 125/200: 14.887952
Init 126/200 with method: k-means++
Inertia for init 126/200: 2.327432
Init 127/200 with method: k-means++
Inertia for init 127/200: 3.779043
Init 128/200 with method: k-means++
Inertia for init 128/200: 4.034629
Init 129/200 with method: k-means++
Inertia for init 129/200: 3.039261
Init 130/200 with method: k-means++
Inertia for init 130/200: 6.098618
Init 131/200 with method: k-means++
Inertia for init 131/200: 8.344823
Init 132/200 with method: k-means++
Inertia for init 132/200: 1.160935
Init 133/200 with method: k-means++
Ine

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 26.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 32.0min finished


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  7.1min finished


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:  1.5min finished


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  7.0min finished


In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    if params['hidden_layer_size'] > 200:
        kmeans = MiniBatchKMeans(n_clusters=200, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
        w_in = np.pad(w_in, ((0, params['hidden_layer_size'] - 200), (0, 0)), mode='constant', constant_values=0)
    else:
        kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
        t1 = time.time()
        kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
        w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
        t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

##  BasicMotions
(http://www.timeseriesclassification.com/description.php?Dataset=BasicMotions)

Data Set Information:

The data was generated as part of a student project where four students performed four activities whilst wearing a smart watch. The watch collects 3D accelerometer and a 3D gyroscope It consists of four classes, which are walking, resting, running and badminton. Participants were required to record motion a total of five times, and the data is sampled once every tenth of a second, for a ten second period.
Attribute Information:

TODO

In [ ]:
walk = np.load(r"E:\multivariate_time_series_dataset\numpy\WALK.npz")
X_train = np.empty(shape=(28, ), dtype=object)
y_train = np.empty(shape=(28, ), dtype=object)
X_test = np.empty(shape=(16, ), dtype=object)
y_test = np.empty(shape=(16, ), dtype=object)

if train:
    for k, (X, y) in enumerate(zip(walk['X'], walk['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.tile(y, (X_train[k].shape[0], 1))
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(walk['Xte'], walk['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.tile(y, (X_test[k].shape[0], 1))
else:
    for k, (X, y) in enumerate(zip(walk['X'], walk['Y'])):
        X_train[k] = X[X.sum(axis=1)!=0, :]  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_train[k] = np.argwhere(y).ravel()
    scaler = StandardScaler().fit(np.concatenate(X_train))
    for k, X in enumerate(X_train):
        X_train[k] = scaler.transform(X=X)  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X

    X_train, y_train = shuffle(X_train, y_train, random_state=0)

    for k, (X, y) in enumerate(zip(walk['Xte'], walk['Yte'])):
        X_test[k] = scaler.transform(X=X[X.sum(axis=1)!=0, :])  # Sequences are zeropadded -> should we remove zeros? if not, X_train[k] = X
        y_test[k] = np.argwhere(y).ravel()

Fit random ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_walk.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_walk.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
base_esn = SeqToLabelESNClassifier(**sequential_search.best_estimator_.get_params())
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    gs_cv = GridSearchCV(clone(base_esn).set_params(**params), 
                         param_grid={'random_state': range(10)}, 
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    t1 = time.time()
    esn = clone(base_esn).set_params(**params).fit(X=X_train, y=y_train, n_jobs=8)
    t2 = time.time()
    score = accuracy_score(y_test, esn.predict(X_test))
    print("ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

Fit KM-ESN (TODO CHECK)

In [ ]:
kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])

initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 0.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-3,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'cv': 2, 'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step2 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step3 = {'cv': 2, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}
kwargs_step4 = {'cv': 2, 'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True)}

# The searches are defined similarly to the steps of a sklearn.pipeline.Pipeline:
searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step4_esn_params, kwargs_step4)]

base_km_esn = SeqToSeqESNClassifier(input_to_node=PredefinedWeightsInputToNode(predefined_input_weights=w_in.T),
                                    **initially_fixed_params)

try:
    sequential_search = load("../sequential_search_walk_km.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_km_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_walk_km.joblib")

In [ ]:
print(sequential_search.all_best_params_)
print(sequential_search.all_best_score_)
"""
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600]}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=20, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=0)
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    base_km_esn = clone(sequential_search.best_estimator_)
    base_km_esn.input_to_node.predefined_input_weights=w_in.T
    base_km_esn.set_params(**params)
    gs_cv = GridSearchCV(base_km_esn,
                         param_grid={'random_state': range(10)},
                         scoring=make_scorer(accuracy_score), n_jobs=-1).fit(X=X_train, y=y_train)
    print(gs_cv.cv_results_)
    print("---------------------------------------")
    acc_score = accuracy_score(y_test, gs_cv.best_estimator_.predict(X_test))
    print(acc_score)
    print("---------------------------------------")
"""
constant_params = sequential_search.best_estimator_.get_params()
constant_params.pop('hidden_layer_size')
constant_params.pop('random_state')
constant_params.pop('predefined_input_weights')
base_esn = SeqToLabelESNClassifier(**constant_params)
param_grid = {'hidden_layer_size': [50, 100, 200, 400, 800, 1600],
              'random_state': range(1, 11)}

for params in ParameterGrid(param_grid):
    kmeans = MiniBatchKMeans(n_clusters=params['hidden_layer_size'], n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=0, random_state=params['random_state'])
    t1 = time.time()
    kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
    w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
    t2 = time.time()
    km_esn = clone(base_esn)
    km_esn.input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
    km_esn.set_params(**constant_params, **params)
    km_esn.fit(X=X_train, y=y_train, n_jobs=8)
    score = accuracy_score(y_test, km_esn.predict(X_test))
    print("KM-ESN with params {0} achieved score of {1} and was trained in {2} seconds.".format(params, score, t2-t1))

In [ ]:
sequential_search = load("../sequential_search_chlo_km.joblib")
sequential_search.all_best_params_, sequential_search.all_best_score_
sequential_search.best_estimator_.input_to_node.input_weights

In [ ]:
# idx = np.random.randint(0, 800, 50)
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.histplot(data=w_in, stat="count", legend=False)
# ax = sns.heatmap(data=sequential_search.best_estimator_.input_to_node.input_weights, cmap="RdBu", vmin=-1.0, vmax=1.0)
# ax.invert_yaxis()
# ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
# ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.xlabel("Neuron Index")
# plt.ylabel("Input Dimension")
plt.xlabel("Weight")
plt.ylabel("Count")
# plt.imshow(sequential_search.best_estimator_.input_to_node.input_weights.todense()[:, idx])
# plt.colorbar()
plt.savefig('KM_ESN_Input_Weight_Hist_CHLO.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])

fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='RdBu', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.97, 1.0)
sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
sm.set_array([])
plt.xlim((0, 2.05))
plt.ylim((0, 1.05))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)
fig.set_size_inches(4, 2.5)
tick_locator = ticker.MaxNLocator(5)
ax.yaxis.set_major_locator(tick_locator)
ax.xaxis.set_major_locator(tick_locator)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Leakage")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(10)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("Score")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
ax.set_xscale('log')
plt.xlabel("Alpha")
plt.ylabel("Score")
plt.xlim((1e-5, 1e0))
tick_locator = ticker.MaxNLocator(20)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()

In [ ]:
sequential_search.best_estimator_

In [ ]:
plt.plot(np.concatenate(X_train))
plt.xlim((0, 1000))